**Họ tên: Phan Anh Lộc**

**Lớp: CS106.M11.KHCL**

**MSSV: 19521766**

In [148]:
import gym
import numpy as np
import random
import time

In [149]:
env1 = gym.make('FrozenLake-v0')
env2 = gym.make('FrozenLake8x8-v0')
env3 = gym.make('Taxi-v3')

In [150]:
# Hyperparameters
gamma = 0.99
learning_rate = 0.1
max_epsilon = 1.0
min_epsilon = 0.01
epsilon_decay_rate = 0.005

num_episodes = 20000
num_steps_per_episode = 100

In [151]:
def q_learning(env, num_episodes, num_steps_per_episode, learning_rate, gamma, max_epsilon, min_epsilon, epsilon_decay_rate):
    q_table = np.zeros((env.observation_space.n, env.action_space.n))
    rewards_all = []
    for episode in range(num_episodes):
        state = env.reset()

        reward_episode = 0.0
        done = False
        epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-epsilon_decay_rate*episode)
        for step in range(num_steps_per_episode):
            exploration = random.uniform(0,1)
            if exploration < epsilon:
                action = env.action_space.sample()
            else:
                action = np.argmax(q_table[state, :])

            next_state, reward, done, info = env.step(action)
            q_table[state, action] = q_table[state, action] * (1 - learning_rate) + learning_rate * (reward + gamma * np.max(q_table[next_state,:]))

            reward_episode += reward
            state = next_state

            if done:
                break
        rewards_all.append(reward_episode)
    print(f'Episode {episode} finished')
    print("Performance : {}".format(sum(rewards_all) / num_episodes))
    return q_table, rewards_all

In [152]:
def sarsa_learning(env, num_episodes, num_steps_per_episode, learning_rate, gamma, max_epsilon, min_epsilon, epsilon_decay_rate):
    q_table = np.zeros((env.observation_space.n, env.action_space.n))
    rewards_all = []
    for episode in range(num_episodes):
        state = env.reset()
        
        reward_episode = 0.0
        done = False
        epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-epsilon_decay_rate*episode)

        # action 1
        exploration = random.uniform(0,1)
        if exploration < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(q_table[state, :])

        for step in range(num_steps_per_episode):
            
            # choose next action
            next_state, reward, done, info = env.step(action)

            # action 2
            exploration = random.uniform(0,1)
            if exploration < epsilon:
                next_action = env.action_space.sample()
            else:
                next_action = np.argmax(q_table[next_state, :])

            # update with Exponentially Weighted Moving Average
            q_table[state, action] = q_table[state, action] + learning_rate * (reward + gamma * q_table[next_state, next_action] - q_table[state, action])

            reward_episode += reward
            state = next_state
            action = next_action

            if done:
                break
        rewards_all.append(reward_episode)
    print(f'Episode {episode} finished')
    print("Performance : {}".format(sum(rewards_all) / num_episodes))
    return q_table, rewards_all

In [153]:
def play(env, q_table, render=False):
    state = env.reset()
    total_reward = 0
    steps = 0
    done = False
    while not done:
        action = np.argmax(q_table[state, :])
        next_state, reward, done, info = env.step(action)
        total_reward += reward
        steps += 1
        if render:
            env.render()
            time.sleep(0.2)
            if not done:
                display.clear_output(wait=True)
        state = next_state

    return (total_reward, steps)

In [154]:
def play_multiple_times(env, q_table, max_episodes):
    success = 0
    list_of_steps = []
    for i in range(max_episodes):
        total_reward, steps = play(env, q_table)

        if total_reward > 0:
            success += 1
            list_of_steps.append(steps)

    print(f'Number of successes: {success}/{max_episodes}')
    print(f'Average number of steps: {np.mean(list_of_steps)}')
    print('Performance: {}'.format(sum(rewards_all) / num_episodes))

#FrozenLake-v0

**Q-LEARNING**

In [159]:
# Initialize Q-value table randomly
q_table = np.zeros((env1.observation_space.n, env1.action_space.n))
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [160]:
q_table, rewards_all = q_learning(env1, num_episodes, num_steps_per_episode, learning_rate, gamma, max_epsilon, min_epsilon, epsilon_decay_rate)

Episode 19999 finished
Performance : 0.63345


In [161]:
q_table

array([[0.50572485, 0.47280769, 0.46774773, 0.44424498],
       [0.30039022, 0.24238206, 0.18553097, 0.43852163],
       [0.33760935, 0.27659418, 0.225413  , 0.26210368],
       [0.1485187 , 0.        , 0.        , 0.        ],
       [0.51938204, 0.35846178, 0.29680517, 0.31524071],
       [0.        , 0.        , 0.        , 0.        ],
       [0.27485379, 0.09483553, 0.18335613, 0.12228992],
       [0.        , 0.        , 0.        , 0.        ],
       [0.45177842, 0.39321581, 0.41068234, 0.55203504],
       [0.43466043, 0.63736052, 0.49782012, 0.36588117],
       [0.60576563, 0.32484168, 0.38210066, 0.17731966],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.59478851, 0.60920646, 0.79094241, 0.60413777],
       [0.75251975, 0.90960793, 0.7570304 , 0.71482777],
       [0.        , 0.        , 0.        , 0.        ]])

In [162]:
sum(rewards_all)

12669.0

In [163]:
sum(rewards_all[0:1000])

228.0

In [164]:
sum(rewards_all[1000:2000])

443.0

In [165]:
sum(rewards_all[2000:3000])

607.0

In [166]:
sum(rewards_all[9000:10000])

661.0

In [167]:
play(env1, q_table)

(1.0, 28)

In [168]:
play_multiple_times(env1, q_table, 1000)

Number of successes: 758/1000
Average number of steps: 37.63192612137203
Performance: 0.63345


**SARSA**

In [169]:
# Initialize Q-value table randomly
q_table = np.zeros((env1.observation_space.n, env1.action_space.n))
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [170]:
q_table, rewards_all = sarsa_learning(env1, num_episodes, num_steps_per_episode, learning_rate, gamma, max_epsilon, min_epsilon, epsilon_decay_rate)

Episode 19999 finished
Performance : 0.63825


In [171]:
q_table

array([[0.52864306, 0.460638  , 0.45356942, 0.44859046],
       [0.23531572, 0.34226069, 0.32021144, 0.47277316],
       [0.40933341, 0.34865466, 0.31405783, 0.38543616],
       [0.23196273, 0.23708509, 0.19981442, 0.37824479],
       [0.54302936, 0.37015753, 0.32887242, 0.34700668],
       [0.        , 0.        , 0.        , 0.        ],
       [0.17131417, 0.17385725, 0.41043101, 0.15788154],
       [0.        , 0.        , 0.        , 0.        ],
       [0.29353014, 0.43839949, 0.32287006, 0.57017555],
       [0.40351913, 0.64361871, 0.47631907, 0.42391173],
       [0.62985958, 0.42040635, 0.44333326, 0.31773129],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.45235656, 0.4584929 , 0.71745383, 0.51530583],
       [0.73747046, 0.86529519, 0.72320547, 0.73912975],
       [0.        , 0.        , 0.        , 0.        ]])

In [172]:
sum(rewards_all)

12765.0

In [173]:
sum(rewards_all[0:1000])

172.0

In [174]:
sum(rewards_all[1000:2000])

381.0

In [175]:
sum(rewards_all[2000:3000])

693.0

In [176]:
sum(rewards_all[9000:10000])

691.0

In [177]:
play(env1, q_table)

(0.0, 33)

In [178]:
play_multiple_times(env1, q_table, 1000)

Number of successes: 742/1000
Average number of steps: 36.27897574123989
Performance: 0.63825


# FrozemLake8x8-v0

**Q-LEARNING**

In [179]:
# Initialize Q-value table randomly
q_table = np.zeros((env2.observation_space.n, env2.action_space.n))
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [180]:
q_table, rewards_all = q_learning(env2, num_episodes, num_steps_per_episode, learning_rate, gamma, max_epsilon, min_epsilon, epsilon_decay_rate)

Episode 19999 finished
Performance : 0.0


In [181]:
q_table

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],


In [182]:
sum(rewards_all)

0.0

In [183]:
sum(rewards_all[0:1000])

0.0

In [184]:
sum(rewards_all[1000:2000])

0.0

In [185]:
sum(rewards_all[2000:3000])

0.0

In [186]:
sum(rewards_all[9000:10000])

0.0

In [187]:
play(env2, q_table)

(0.0, 200)

In [188]:
play_multiple_times(env2, q_table, 1000)

Number of successes: 0/1000
Average number of steps: nan
Performance: 0.0


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


**SARSA**

In [189]:
# Initialize Q-value table randomly
q_table = np.zeros((env2.observation_space.n, env2.action_space.n))
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [190]:
q_table, rewards_all = sarsa_learning(env2, num_episodes, num_steps_per_episode, learning_rate, gamma, max_epsilon, min_epsilon, epsilon_decay_rate)

Episode 19999 finished
Performance : 0.0


In [191]:
q_table

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],


In [192]:
sum(rewards_all)

0.0

In [193]:
sum(rewards_all[0:1000])

0.0

In [194]:
sum(rewards_all[1000:2000])

0.0

In [195]:
sum(rewards_all[2000:3000])

0.0

In [196]:
sum(rewards_all[9000:10000])

0.0

In [197]:
play(env2, q_table)

(0.0, 200)

In [200]:
play_multiple_times(env2, q_table, 1000)

Number of successes: 0/1000
Average number of steps: nan
Performance: 0.0


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


# Taxi-v3

**Q-LEARNING**

In [225]:
# Initialize Q-value table randomly
q_table = np.zeros((env3.observation_space.n, env3.action_space.n))
print(q_table)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


In [226]:
q_table, rewards_all = q_learning(env3, num_episodes, num_steps_per_episode, learning_rate, gamma, max_epsilon, min_epsilon, epsilon_decay_rate)

Episode 19999 finished
Performance : 0.1487


In [227]:
q_table

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-0.09874413, -2.69174362, -0.11786845,  0.33844417,  9.6220697 ,
        -6.73437877],
       [ 0.38835932,  0.63094994,  4.17686585,  2.93596218, 14.11880599,
        -5.70611527],
       ...,
       [-0.87762416, -0.47009482, -0.89543415,  3.86450163, -1.        ,
        -1.        ],
       [-2.42842008, -2.42952754, -2.36459818, -2.36199705, -4.94339487,
        -5.46671191],
       [-0.1       , -0.1       , -0.19      , 11.97292607,  0.        ,
        -1.        ]])

In [228]:
sum(rewards_all)

2974.0

In [229]:
sum(rewards_all[0:1000])

-128123.0

In [230]:
sum(rewards_all[1000:2000])

-1645.0

In [231]:
sum(rewards_all[2000:3000])

6842.0

In [232]:
sum(rewards_all[9000:10000])

7418.0

In [233]:
play(env3, q_table)

(11, 10)

In [234]:
play_multiple_times(env3, q_table, 1000)

Number of successes: 1000/1000
Average number of steps: 13.108
Performance: 0.1487


**SARSA**

In [235]:
# Initialize Q-value table randomly
q_table = np.zeros((env3.observation_space.n, env3.action_space.n))
print(q_table)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


In [236]:
q_table, rewards_all = sarsa_learning(env3, num_episodes, num_steps_per_episode, learning_rate, gamma, max_epsilon, min_epsilon, epsilon_decay_rate)

Episode 19999 finished
Performance : 0.1561


In [237]:
q_table

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-2.38140893, -2.04682222, -2.66492113, -0.34982369,  8.94859995,
        -5.2581628 ],
       [ 1.72375436, -1.31523103, -3.14689921, -2.99339191, 13.88614973,
        -6.16999955],
       ...,
       [-1.93920598, -1.80646989, -2.01388151, -1.93853173, -2.76775537,
        -2.92021015],
       [-3.57840731, -3.56706577, -3.62265634, -3.56923992, -6.27507433,
        -4.91289631],
       [-1.49631353, -0.77010621, -0.58740478, 11.97398657, -3.6651363 ,
        -5.78804823]])

In [238]:
sum(rewards_all)

3122.0

In [239]:
sum(rewards_all[0:1000])

-126510.0

In [240]:
sum(rewards_all[1000:2000])

-4246.0

In [241]:
sum(rewards_all[2000:3000])

7098.0

In [242]:
sum(rewards_all[9000:10000])

7369.0

In [243]:
play(env3, q_table)

(10, 11)

In [244]:
play_multiple_times(env3, q_table, 1000)

Number of successes: 1000/1000
Average number of steps: 13.048
Performance: 0.1561


# Đánh giá performance

Ở 2 thuật toán Q-learning và Sarsa-learning thì em đánh giá performance bằng cách lấy tổng số điểm thưởng chia cho số episode



# Nhận xét

Cả hai thuật toán Sarsa và Q-learning đều không chạy được trên map FrozenLake8x8-v0

Xét về tiêu chí tổng điểm thưởng thì Sarsa cho điểm cao hơn Q-learning, cụ thể:
+ Ở map FrozenLake-v0, Sarsa cho tổng điểm thưởng là 12765. Trong khi đó của Q-learning là 12669
+ Ở map Taxi-v3, Sarsa cho tổng điểm thưởng là 3122. Trong khi đó của Q-learning là 2974

Xét về tiêu chí số bước đi trung bình thì Sarsa cho số bước đi ít hơn, cụ thể:
+ Ở map FrozenLake-v0, Sarsa cho số bước đi trung bình là 36.27. Trong khi đó của Q-learning là 37.63
+ Ở map Taxi-v3, Sarsa cho số bước đi trung bình là 13.048. Trong khi đó của Q-learning là 13.108

Xét về tiêu chí performance thì Sarsa vẫn làm tốt hơn Q-learning, cụ thể:
+ Ở map FrozenLake-v0, Sarsa cho kết quả performance là 0.63825. Trong khi đó của Q-learning là 0.63345
+ Ở map Taxi-v3, Sarsa cho số bước đi trung bình là 0.1561. Trong khi đó của Q-learning là 0.1487

Tổng quan lại, thuật toán Sarsa cho kết quả performance tốt hơn Q-learning ở tất cả các map